# Bloque 4 · De RAG a Chatbot: La Capa de Presentación

En los bloques anteriores exploramos los componentes fundamentales de RAG: comprendimos el flujo completo, simulamos la búsqueda vectorial, aprendimos a invocar knowledge bases en Bedrock y utilizamos la API de RAG para generar respuestas con LLM. Este bloque se enfoca en cómo integrar toda esa infraestructura en una interfaz conversacional que permita a los usuarios interactuar de forma natural con el sistema RAG.

## La necesidad de una interfaz conversacional

Una vez que tenemos un sistema RAG funcional capaz de recuperar información relevante y generar respuestas coherentes, el siguiente paso natural es exponerlo a través de una interfaz que permita a los usuarios interactuar de manera intuitiva. Un chatbot conversacional es la forma más natural de presentar un sistema RAG porque:

- **Interacción natural**: Los usuarios pueden hacer preguntas en lenguaje natural, similar a cómo interactuarían con un experto humano
- **Feedback inmediato**: Las respuestas se generan en tiempo real, permitiendo una conversación fluida
- **Transparencia**: Puede mostrar las fuentes y citas que respaldan cada respuesta, generando confianza
- **Escalabilidad**: Un solo chatbot puede atender múltiples usuarios simultáneamente

## ¿Qué es Chainlit?

Chainlit es un framework de Python diseñado específicamente para construir interfaces conversacionales con modelos de IA. Su filosofía se basa en:

- **Simplicidad**: Permite crear chatbots web funcionales con mínima configuración, sin necesidad de construir un front-end desde cero
- **Orientación a flujos conversacionales**: Facilita el manejo del estado de la conversación, la presentación de componentes ricos (citas, tablas, visualizaciones) y el registro de interacciones
- **Productividad**: Permite iterar rápidamente sobre la experiencia del usuario mientras se sigue afinando el backend RAG
- **Integración sencilla**: Utiliza decoradores simples (`@cl.on_chat_start`, `@cl.on_message`) para definir el comportamiento del chatbot

En nuestra arquitectura, Chainlit ocupa la **capa de presentación**: recibe la pregunta del usuario, invoca nuestras funciones Python (que a su vez llaman a Bedrock), y presenta la respuesta final junto con las citas correspondientes de manera visual y accesible.

## Arquitectura del flujo completo

El flujo de un chatbot RAG con Chainlit sigue una arquitectura en capas bien definida:

### 1. Capa de presentación (Chainlit)
- **Recepción de mensajes**: El usuario escribe una pregunta en la interfaz web
- **Gestión de sesión**: Chainlit mantiene el contexto de la conversación
- **Renderizado de respuestas**: Presenta el texto generado y las citas de forma visual

### 2. Capa de orquestación (Python)
- **Procesamiento de entrada**: Recibe la pregunta del usuario desde Chainlit
- **Invocación de RAG**: Llama a las funciones que implementan el flujo RAG (recuperación + generación)
- **Procesamiento de salida**: Extrae el texto generado y las citas de la respuesta de Bedrock
- **Formateo**: Prepara la información para su presentación en Chainlit

### 3. Capa de servicios (AWS Bedrock)
- **Recuperación**: Busca fragmentos relevantes en la knowledge base usando búsqueda vectorial
- **Generación**: El LLM sintetiza una respuesta basada en los fragmentos recuperados
- **Trazabilidad**: Devuelve citas que vinculan cada parte de la respuesta con sus fuentes

### Flujo de datos

```
Usuario → Chainlit → Función Python → Bedrock API → Knowledge Base
                                                          ↓
Usuario ← Chainlit ← Función Python ← Respuesta + Citas ←
```

Este diseño desacopla las responsabilidades: la lógica RAG permanece independiente de la interfaz, lo que facilita experimentar con diferentes frameworks de UI, cambiar modelos, o agregar nuevas funcionalidades sin reescribir todo el sistema.

## Componentes clave de un chatbot RAG

Para construir un chatbot que integre RAG, necesitamos combinar varios componentes:

### Funciones de RAG (del Bloque 3)
- **`generar_con_prompt()`**: Encapsula la lógica de invocación a `retrieve_and_generate` de Bedrock
- **`mostrar_generacion_simple()`**: Versión adaptada que extrae texto y citas para presentación en UI
- **`DEFAULT_PROMPT_TEMPLATE`**: Template que guía al LLM sobre cómo usar el contexto recuperado

### Handlers de Chainlit
- **`@cl.on_chat_start`**: Se ejecuta cuando un usuario inicia una sesión. Ideal para:
  - Inicializar recursos (clientes de AWS, cargar configuración)
  - Mostrar mensaje de bienvenida
  - Establecer el contexto inicial de la conversación

- **`@cl.on_message`**: Se ejecuta por cada mensaje del usuario. Debe:
  - Extraer la pregunta del mensaje
  - Invocar la función RAG con los parámetros apropiados
  - Procesar la respuesta y extraer texto y citas
  - Enviar la respuesta formateada de vuelta al usuario

### Configuración y variables
- **Variables de entorno**: Para mantener configuración flexible (knowledge base ID, modelo ARN, región AWS)
- **Credenciales AWS**: Autenticación para acceder a Bedrock (pueden venir de variables de entorno, archivos de credenciales, o roles IAM)

## Flujo de ejecución del chatbot

Cuando un usuario interactúa con el chatbot, se ejecuta el siguiente flujo:

### Inicio de sesión (`on_chat_start`)
1. Chainlit detecta que un usuario abre el chat
2. Se ejecuta la función decorada con `@cl.on_chat_start`
3. Se inicializan recursos (si es necesario) y se envía mensaje de bienvenida
4. El usuario ve la interfaz lista para recibir preguntas

### Procesamiento de mensajes (`on_message`)
1. **Recepción**: El usuario escribe una pregunta y Chainlit captura el mensaje
2. **Indicador de carga**: Se muestra un mensaje temporal "Procesando..." para dar feedback inmediato
3. **Invocación RAG**: Se llama a `generar_con_prompt()` con la pregunta del usuario
4. **Procesamiento en Bedrock**:
   - La pregunta se convierte en embedding
   - Se buscan fragmentos similares en la knowledge base
   - El LLM genera una respuesta usando esos fragmentos
   - Se devuelven citas que vinculan la respuesta con las fuentes
5. **Extracción**: `mostrar_generacion_simple()` procesa la respuesta y separa texto de citas
6. **Presentación**: Se envían dos mensajes:
   - La respuesta principal del asistente
   - Las fuentes consultadas (si existen)

### Manejo de errores
Es importante incluir manejo de excepciones para:
- Errores de conexión con AWS
- Timeouts en la API
- Respuestas vacías o inválidas
- Problemas de autenticación

Esto garantiza que el usuario siempre reciba feedback, incluso cuando algo falla.

## Resumen: Arquitectura completa de extremo a extremo

Hemos recorrido todo el flujo de un sistema RAG completo:

### Capas del sistema

1. **Capa de datos** (Bloque 1)
   - Documentos fuente almacenados en S3
   - Knowledge base con embeddings vectoriales
   - Metadatos para trazabilidad

2. **Capa de recuperación** (Bloque 2)
   - Búsqueda vectorial semántica
   - Ranking por similitud
   - Extracción de fragmentos relevantes

3. **Capa de generación** (Bloque 3)
   - LLM que sintetiza respuestas
   - Prompt engineering para guiar el comportamiento
   - Generación de citas y trazabilidad

4. **Capa de presentación** (Bloque 4)
   - Interfaz conversacional con Chainlit
   - Visualización de respuestas y fuentes
   - Gestión de sesiones y estado

### Principios de diseño

- **Desacoplamiento**: Cada capa es independiente y puede evolucionar por separado
- **Reutilización**: Las funciones de RAG pueden usarse en múltiples contextos (notebooks, APIs, chatbots)
- **Trazabilidad**: Siempre sabemos qué fuentes respaldan cada respuesta
- **Flexibilidad**: Fácil cambiar modelos, prompts, o interfaces sin reescribir todo

Este patrón arquitectónico permite construir sistemas RAG robustos, escalables y mantenibles que pueden adaptarse a diferentes necesidades y evolucionar con el tiempo.

## Actividad Práctica

En esta sección vamos a trabajar paso a paso para construir un chatbot RAG completo con Chainlit. Vamos a partir de una versión mínima funcional y la vamos a ir mejorando hasta llegar a una implementación más completa.

### Estructura de los ejercicios

Los ejercicios están diseñados en orden de complejidad creciente. Cada ejercicio agrega funcionalidad sobre el anterior, permitiendo que entiendas cada componente antes de avanzar al siguiente nivel.

**Archivo de referencia:**
- `chatbot/chatbot_chainlit.py`: Versión mínima funcional (punto de partida)

---

### Ejercicio 1: Configurar entorno y ejecutar el bot básico

**Objetivo:** Configurar el entorno de desarrollo y ejecutar la versión mínima del chatbot.

**Pasos:**

1. **Creá un entorno virtual:**
   ```bash
   python -m venv taller-rag
   ```

2. **Activá el entorno virtual:**
   - En Windows: `taller-rag\Scripts\activate`
   - En Linux/Mac: `source taller-rag/bin/activate`

3. **Instalá dependencias desde `requirements.txt`:**
   ```bash
   pip install -r chatbot/requirements.txt
   ```

4. **Revisá el código base:**
   abrí `chatbot/chatbot_chainlit.py` y familiarizate con su estructura:
   - Configuración de AWS
   - Función `generar_con_prompt()`
   - Handlers de Chainlit (`on_chat_start`, `on_message`)

5. **Configurar credenciales AWS:**
   Por ahora, vamos a hardcodear las credenciales directamente en el código (en el ejercicio final las refactorizamos). Editá `chatbot/chatbot_chainlit.py` y reemplazá los valores de `os.getenv()` con tus credenciales reales:
   ```python
   # TODO: Reemplazar los valores de os.getenv() con tus credenciales reales
   # Por ahora hardcodeamos las credenciales (en "Pasos a seguir" las refactorizamos)
   session = boto3.Session(
       aws_access_key_id="TU_ACCESS_KEY_ID",  # Reemplazá con tu access key ID
       aws_secret_access_key="TU_SECRET_ACCESS_KEY",  # Reemplazá con tu secret access key
       aws_session_token="TU_SESSION_TOKEN"  # Reemplazá con tu session token (si usás credenciales temporales)
   )
   
   # También podés verificar que las constantes estén configuradas:
   # AWS_REGION = "us-west-2"  # o la región que corresponda
   # KNOWLEDGE_BASE_ID = "TU_KB_ID"  # Tu Knowledge Base ID
   # MODEL_ARN = "us.deepseek.r1-v1:0"  # El modelo que querés usar
   ```
   
   **💡 Pista:** si no tenés credenciales AWS, podés obtenerlas desde la consola de AWS o usando el comando `aws configure` si tenés AWS CLI instalado.

6. **Ejecutá el chatbot:**
   ```bash
   chainlit run chatbot/chatbot_chainlit.py -w
   ```

7. **Probá el chatbot:**
   - abrí el navegador en la URL que Chainlit muestra (típicamente `http://localhost:8000`)
   - hacé algunas preguntas sobre RAG y verificá que el bot responda correctamente

**✅ Criterio de éxito:** El chatbot se ejecuta sin errores y responde preguntas básicas.

---

### Ejercicio 2: Extraer y mostrar citas básicas

**Objetivo:** extraer información de citas de la respuesta de Bedrock y mostrarlas al usuario.

**Tareas:**

1. **Creá la función `extraer_citas_completas()`:** esta función tiene que procesar el campo `citations` de la respuesta de Bedrock y extraer:
   - El texto citado (span)
   - Las referencias (fuentes) que respaldan cada cita
   - Las URIs de las fuentes

2. **Modificá `on_message` para mostrar citas:** después de mostrar la respuesta principal, mostrá las fuentes consultadas.

3. **Formato básico de citas:** por ahora, mostrá las citas como una lista simple de URIs o texto plano. En el siguiente ejercicio vamos a mejorar la visualización.

**💡 Pista:** revisá la estructura de la respuesta de `retrieve_and_generate` en el Bloque 3 para entender cómo están organizadas las citas.

**✅ Criterio de éxito:** El chatbot muestra las fuentes que respaldan cada respuesta.

---

### Ejercicio 3: Agregar logging básico

**Objetivo:** implementar logging para facilitar el debugging y monitoreo del chatbot.

**Tareas:**

1. **Configurá logging al inicio del archivo:**
   ```python
   import logging
   
   logging.basicConfig(
       level=logging.INFO,
       format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
       handlers=[
           logging.FileHandler('chatbot.log', encoding='utf-8'),
           logging.StreamHandler()
       ]
   )
   logger = logging.getLogger(__name__)
   ```

2. **Agregá logs en puntos clave:**
   - al iniciar sesión (`on_chat_start`)
   - al recibir una pregunta (`on_message`)
   - antes y después de llamar a Bedrock (`generar_con_prompt`)
   - al procesar errores

3. **Probá y revisá los logs:** ejecutá el chatbot y verificá que los logs se escriban correctamente en `chatbot.log` y en la consola.

**✅ Criterio de éxito:** Los logs capturan información relevante sobre el flujo de ejecución y errores.

---

### Ejercicio 4: Integrar componente JSX para citas interactivas

**Objetivo:** integrar el componente JSX existente en el chatbot para mostrar citas de forma elegante e interactiva.

#### ¿Qué es JSX?

JSX (JavaScript XML) es una extensión de sintaxis de JavaScript que permite escribir código que se parece a HTML dentro de JavaScript. En el contexto de Chainlit, los componentes JSX permiten crear elementos visuales personalizados para la interfaz del chatbot usando React.

Un componente JSX es básicamente una función que recibe datos (llamados "props") y retorna elementos visuales. Por ejemplo, el componente `Citations.jsx` que ya existe en `chatbot/public/elements/Citations.jsx` recibe una lista de citas y las muestra en un formato interactivo con acordeones desplegables.

**Tareas:**

1. **Revisá el componente existente:** el archivo `chatbot/public/elements/Citations.jsx` ya contiene un componente React que muestra las citas de forma interactiva. Abrí el archivo y familiarizate con su estructura para entender qué datos espera recibir.

2. **Integrá el componente en Python:** modificá tu función `on_message` para usar `cl.CustomElement` y pasar las citas al componente JSX:
   ```python
   # Después de extraer las citas con extraer_citas_completas()
   if citas_completas:
       citations_element = cl.CustomElement(
           name="Citations",
           props={"citations": citas_completas}
       )
       await cl.Message(
           content="",
           elements=[citations_element],
           author="Asistente RAG"
       ).send()
   ```

3. **Verificá el formato de datos:** asegurate de que `citas_completas` tenga el formato correcto que espera el componente JSX. Cada cita debe tener:
   - `citation_index`: número de la cita
   - `texto_citado`: el texto que está siendo citado
   - `span_start` y `span_end`: posiciones del span
   - `referencias`: lista de referencias con `source` y `content`

4. **Probá la visualización:** ejecutá el chatbot y verificá que las citas se muestren correctamente en formato colapsable y que se puedan expandir para ver detalles.

**💡 Pista:** el componente JSX espera recibir las citas en el mismo formato que retorna `extraer_citas_completas()`, así que solo necesitás pasarle directamente el resultado de esa función.

**✅ Criterio de éxito:** Las citas se muestran en un formato visual interactivo y profesional usando el componente JSX.

---

## Pasos a seguir

Una vez completados los ejercicios anteriores, te recomendamos implementar estas mejoras adicionales para hacer tu chatbot más robusto y profesional:

### Variables de entorno

**Objetivo:** mejorar la seguridad y flexibilidad del código usando variables de entorno en lugar de credenciales hardcodeadas.

**Pasos:**

1. **Identificá los valores hardcodeados:**
   - Credenciales AWS (access key, secret key, session token)
   - Knowledge Base ID
   - Model ARN
   - AWS Region

2. **Refactorizá para usar `os.getenv()`:** agregá código para leer las variables de entorno en lugar de valores hardcodeados:
   ```python
   AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
   AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
   AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")
   KNOWLEDGE_BASE_ID = os.getenv("BEDROCK_KB_ID")
   MODEL_ARN = os.getenv("BEDROCK_MODEL_ARN")
   AWS_REGION = os.getenv("AWS_REGION", "us-west-2")
   ```

3. **Creá un archivo `.env` de ejemplo:** documentá qué variables de entorno son necesarias.

4. **Actualizá la documentación:** agregá instrucciones sobre cómo configurar las variables de entorno.

### Manejo de errores

**Objetivo:** agregar manejo de errores más específico y mensajes informativos para el usuario.

**Pasos:**

1. **Identificá tipos de errores comunes:**
   - Errores de autenticación AWS
   - Errores de conexión/timeout
   - Errores de la API de Bedrock
   - Respuestas vacías o inválidas

2. **Modificá `on_message` para manejar errores específicos:**
   ```python
   @cl.on_message
   async def on_message(message: cl.Message):
       try:
           respuesta = generar_con_prompt(message.content)
           texto = respuesta.get("output", {}).get("text", "<Sin respuesta>")
           
           if not texto or texto == "<Sin respuesta>":
               await cl.Message(content="❌ No se pudo generar una respuesta válida.", author="Sistema").send()
               return
               
           await cl.Message(content=texto, author="Asistente RAG").send()
       except cliente.exceptions.ClientError as e:
           # Manejar errores específicos de AWS
           await cl.Message(content=f"❌ Error de AWS: {str(e)}", author="Sistema").send()
       except Exception as e:
           # Manejar otros errores
           await cl.Message(content=f"❌ Error: {str(e)}", author="Sistema").send()
   ```

3. **Agregá validación de respuesta vacía:** verificá que la respuesta contenga texto antes de enviarla al usuario.